# Data Science Summer Intern assignment 2021

Rasmus Siljander, 18.1.2021


## Time Series Analysis

This notebook contains my approach to the Wolt Data Science Summer Intern assignment. The notebook focuses on the time
series analysis of daily Wolt orders. More specifically, I try to build different SARIMA models that could help predict
the number of Wolt orders in the near future.

#### Installing packages


In [ ]:
import warnings

import pandas as pd
import numpy as np
from random import randint

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

from collections import Counter
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


### The Data

First, its important to take a closer look at the data. We have the assignment data saved in the .csv file
**orders_autumn_2020.csv** (in the same directory as this Jupyter Notebook)


In [ ]:
data = pd.read_csv('orders_autumn_2020.csv', parse_dates=['TIMESTAMP'])
data.head()


### Feature: Number of orders per day

In terms of time series analysis, the key feature that we are focused on is daily order frequency: being able to
predict order frequency and size allows for better planning and allocation of couriers. This results in better demand
 anticipation.

Intuitively, we should expect seasonal spikes in the time series during
weekends - during friday, saturday, and sunday people tend to spend time with their loved ones, hang out, or just stay in,
relax, and just order something nice from Wolt. We shall evaluate this by inspecting the time series as well as the
seasolly differenced series:

####  Preparing data


In [ ]:
date = data['TIMESTAMP'].dt.round(freq = 'D')

frequencies = dict(Counter(date))
dates = frequencies.keys()
orders = pd.Series(frequencies.values(), index = dates)       #converted
orders_diff7 = orders.diff(periods = 7)


#### Plotting time series'

In [ ]:
fig, ax = plt.subplots(2,1, figsize = (10,9))
ax[0].plot(dates, orders)

date_format = DateFormatter('%d.%m.%Y')
ax[0].xaxis.set_major_formatter(date_format)

ax[0].set_ylabel('Orders')
ax[0].set_title('Original time series')

ax[1].plot(orders_diff7)
ax[1].xaxis.set_major_formatter(date_format)
ax[1].set_xlabel('Date')

ax[1].set_ylabel('7-day Difference in orders')
ax[1].set_title('7-day Differenced time series')


plt.show()


We can see from the figure that this suspicion seems correct: there are seasonal spikes approximately every seven days,
and the differenced time series seems to be more stationary. The first observation, 8.8.2020, seems to be bit outlying
from the rest of the time series, but we will leave it for now. What's also interesting is the significant spike in
orders in late August, as the weekend-increase in orders for this date is significantly higher than all other weekends.
It would be interesting to go back and investigate, if some significant event (schools started, major lockdown) occured
during that time that could help explaine this spike.

Overall, since the differenced time series seems to be relatively stationary (no distinguishable trend, no changes in
variance or seasonality), it is possible that a simple seasonal SARIMA could possibly be
able to model this time series. First, we could plot the (partial) autocorrelation functions to give a better idea of
what SARIMA model fits best. These help us estimate the parameters for our SARIMA model.

The SARIMA model is composed of  seven parameters: the autoregressive, difference, and moving
average components of the time series. Although this is not an exact science, and different parameter combinations can
work as satisfactory predictors, we can estimate SARIMA parameters from the ACF and PACF plots. (check this link or any
relevant google article  for more information: https://machinelearningmastery.com/sarima-for-time-series-forecasting-in-python/)

In [ ]:
plot_acf(orders, lags = 50)
plot_pacf(orders, lags = 30)
plt.show()


The first figure shows that the ACF of series seems to decay exponentially at 1s, 2s, 3s, ... where s = 7 (days).
This suggests that the autoregressive components of the time series could be 1. The PACF is a bit more inconclusive,
although again it seems that PACF cuts off after 7 (=1s). This, again, would indicate that we have a moving average
component of 1. Aggregating these results, it seems that a plausible model could be $SARIMA (1,1,1) (1,1,1)_7$, although
other parameter combinations are possible. We shall first test the proposed model and evaluate its effectiveness:

In [ ]:
warnings.filterwarnings('ignore')   #ignore ValueWarning
model = SARIMAX(orders, order = (1,1,1), seasonal_order = (1,1,1,7))
fit = model.fit()
print(f'AIC:{fit.aic:.2f}')
warnings.filterwarnings('default')

Above we have also included the Akaike Information Criterion (AIC), an advanced model evaluation statistic. This value
can be used as reference for the other models to evaluate performance (the lower the better). Fitting the predictions
visualizing the residuals:

In [ ]:
ts_fit = fit.fittedvalues
pred_fit = fit.forecast(steps = 30)
res_fit = fit.resid

In [ ]:
fig, ax = plt.subplots(figsize = (10,8))
ax.plot(res_fit)
ax.xaxis.set_major_formatter(date_format)
ax.axhline(0, c = 'black', linestyle = '--')
plt.show()

It seems that the model residuals are relatively uniform (with some extreme outliers). Overall, based on this plot, it
seems that the model is satisfactory, and that there is no outlying trend that could be visible from the plot. We can
next plot the model with the time series to evaluate mode fit:

In [ ]:
fig, ax = plt.subplots(figsize = (10,7))
date_format = DateFormatter('%d.%m.%Y')

ax.plot(orders, label = 'Original Data')
ax.plot(ts_fit, label = 'SARIMA(1,1,1)(1,1,1)7')
ax.plot(pred_fit, label = '30-day prediction')

ax.set_xlabel('Date')
ax.set_ylabel('Orders')
ax.xaxis.set_major_formatter(date_format)
ax.set_title('Orders time series')

plt.legend()
plt.show()

As we can see, although the model seems to moderately well fit the time series data, there is poor long-term
predictability (then again, this is expected, as SARIMA models are not known as accurate long-term predictors).
 Visually evaluating, the model can predict orders for the next seven days, after which the
negative trend of the model results in model inaccuracy.

To combat this, we could simulate SARIMA parameters and choose the most appropriate model. Due to its
easy availability from the fit-object, the Akaike Information Criterion (AIC) is used to evaluate model quality. This
might not be the most efficient or correct method of evaluating model accuracy, but since we are only looking for
preliminary tools for evaluating model performance (and we are going to use other metrics), this statistic was chosen
for its convenience.

The following loop randomly samples all parameters to be between 0 and 2, fits a SARIMA model on them, and stores the
results in an array or list. These are then used for further analysis.


In [ ]:
n_sample = 30
fits = [None]*n_sample
params = np.zeros((n_sample,6))

warnings.filterwarnings('ignore')
for i in range(n_sample):
    params[i] = [randint(0,2) for i in range(6)]
    try:
        model = SARIMAX(orders, order = (params[i,0],params[i,1],params[i,2]),
                        seasonal_order = (params[i,3],params[i,4],params[i,5],7))
        fits[i] = model.fit()

    except np.linalg.LinAlgError:             # just in case there is some approximation
        print('LINALG ERROR')                 # errors with impossible parameters
        pass
aics = [fit.aic for fit in fits if fit.aic > 0]
warnings.filterwarnings('default')

We can check the simulated AIC values in the plot below:

In [ ]:
plt.plot(range(n_sample), aics)
plt.xlabel('Iteration number')
plt.ylabel('AIC')
plt.title('AIC vs parameter combination')
plt.show()

From the randomly simulated models, we can take the three models with the lowest AIC and evaluate them on the original time series:


In [ ]:
id = np.argsort(aics)[:3]       #chooses three array indexes with smallest AIC.
params_id = params[id]
print('Best parameters:\n',params_id)

fit1 = fits[id[0]]
fit2 = fits[id[1]]
fit3 = fits[id[2]]

ts_fit1 = fit1.fittedvalues
pred_fit1 = fit1.forecast(steps = 30)
res_fit1 = fit1.resid

ts_fit2 = fit2.fittedvalues
pred_fit2 = fit2.forecast(steps = 30)
res_fit2 = fit2.resid

ts_fit3 = fit3.fittedvalues
pred_fit3 = fit3.forecast(steps = 30)
res_fit3 = fit3.resid


From the residual plots as well as the AIC plot, it seems that these two models are relatively similar. We can evaluate
their fit and predictability by plotting them with the original time series:

In [ ]:
def lbl_construct(params_list):  #helper function for legend labels
    return 'SARIMA({:.0f},{:.0f},{:.0f})({:.0f},{:.0f},{:.0f})7'.format(*params_list)

In [ ]:
fig, ax = plt.subplots(3,1, figsize = (10,11))

ax[0].plot(orders, label = 'Original Data')

lbl = lbl_construct(params_id[0])
ax[0].plot(ts_fit1, label = lbl)
ax[0].plot(pred_fit1, label = '30-day prediction')

ax[0].xaxis.set_major_formatter(date_format)

ax[0].set_ylabel('Orders')
ax[0].set_title('Orders time series')
ax[0].legend()


lbl = lbl_construct(params_id[1])
ax[1].plot(orders, label = 'Original Data')
ax[1].plot(ts_fit2, label = lbl)
ax[1].plot(pred_fit2, label = '30-day prediction')

ax[1].set_xlabel('Date')
ax[1].set_ylabel('Orders')

ax[1].legend()

lbl = lbl_construct(params_id[2])
ax[2].plot(orders, label = 'Original Data')
ax[2].plot(ts_fit3, label = lbl)
ax[2].plot(pred_fit3, label = '30-day prediction')

ax[2].set_xlabel('Date')
ax[2].set_ylabel('Orders')

ax[2].legend()

plt.show()

From the plots above we can see that there is hardly any help with the randomly iterated models. It therefore seems that
the SARIMA models are, as it stands, inadequate in being able to accurately predict daily orders.


## Evaluation

This exploration fitted a SARIMA model in an attempt to explain and predict Wolt daily order data. Four different SARIMA
models with four parameter combinations were attempted. One model was a proposed model based on ACF and PACF plots, and
the latter three were simulated and chosen from iterations and AIC plots. The simulated models were not found to be
significantly better.

### Improvements

#### Data set
One of the possible issues was the size of the per-day time series: although the original data set had over 18700
observations, the transformed time series had data from 62 different days. This compression of data meant that it was
more difficult to construct precise models with simple processes. Therefore, to improve these models I could look into
either a)growing the time series data set b) look into **models on untransformed data** c) attempt to remove noise and
outliers from the time series. I believe that having order data from other months could help build a better model, but
more importantly, I could try to use the 18000 observations that I already have to my advantage. For instance, instead
of modeling daily orders, I could have modeled **items** ordered by time stamp instead of day. This would have kept the
original time series intact and allowed me to inspect a large data series per ordered items. (orders/day conversion
would have also been easy afterwards)

#### Other model options

**Dynamic Regression**

One of the model improvements that could have been done are dynamic regression models, where our explanatory variables
might be autocorrelated with itself from different time frames.

**Machine Learning model**

One of the other model options that I was thinking of implementing was machine learning models, one built with, for
example, Tensorflow/Keras. I did not implement this myself (since this is already quite a long report), but here are
interesting links that I would have used for my implementation:

https://www.tensorflow.org/tutorials/structured_data/time_series
https://codeit.us/blog/machine-learning-time-series-forecasting
https://medium.com/analytics-steps/introduction-to-time-series-analysis-time-series-forecasting-machine-learning-methods-models-ecaa76a7b0e3

**Stochastic Model**

The last model option (this is in the "only a cool idea"-category) would be to build a stochastic model for the time
between order. If we would have collected all the times between orders and fit them to a particular distribution, we
could possibly be able to come up with model likelihoods for how long Wolt would have to wait between orders. This,
however, does not introduce accurate predictability.

## Your Background and Wolt

I am an extremenly curious and data-orientied person. My interest in data-driven solutions are complemented by my minor
in Data Science, and throughout my (short) career, I have found myself working on
various problems, investigations, and reports from which I have learned significantly.

The most noteworthy project that I have started working on is a project course, in which we are using **machine learning models
(Tensorflow/Keras) for MRI image conversion**. This is a fascinating opportunity for me to learn how to use efficient
machine learning models for image recognition, classification and conversion.

A very important learning process for me was a textual analysis software, in which we used **Regex-based textual analysis
tools to evaluate environmental disclosures**. This software was evaluated in a Master's Thesis for the University of Oulu,
and it was exciting for me to partake in the entirety of the software development process.

Other projects that have allowed me for personal development are logistic regression models for autoantibody analysis
and Bayesian models for basketball shooting. These have taught me how to efficiently manipulate data, analyze it, and
develop efficient analytical tools for meaningful inference.


## Work interests

I would be coming to Wolt with a fresh mindset and fervour for exploring new ideas. Since all of three of the proposed
assignment topics really excited me, I am sure that I would be motivated with anything that Wolt would offer me. I am
interested in working with data, orders, people, images, or applications where I could work on my skills and help create
something exciting.

The image classification assignment also really excited me, and I would love to explore that as a project further. I
could be a part of the team that works on analyzing and classifying food images from the restaurants, and reporting
any appropriate information, be it food type, dish type, dish quality or amount of fresh produce involved - the list of
image classification techniques go on and on.

Alternatively, exploring maps and networks are fascinating in their own right, and so are prediciton models for orders,
location and more!

I would love to further discuss any open positions that you could offer to me in this field.